# Exercise Week 9, Lecture 17 -- Guided

In [1]:
# imports
import numpy as np
from scipy.ndimage import uniform_filter1d
from scipy.stats import shapiro, bartlett
from matplotlib import pyplot as plt
import pandas

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import pacf

import pymannkendall as mk
# EVA
from pyextremes import EVA

# Load up the data

## ENSO

In [11]:
months =['Jan', 'Feb', 'Mar', 'Apr', 'May',
                                  'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
enso_file = '../Data/enso_soi_ts.txt'
enso_df = pandas.read_table(enso_file, delim_whitespace=True,
                           names=['Year']+months)
enso_df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1870,-0.94,-0.80,-0.25,-0.76,-0.91,-0.49,-0.75,-1.07,-0.26,-0.26,0.03,0.23
1,1871,0.39,-0.28,0.52,-0.16,-0.22,-0.45,-0.46,-0.39,-0.33,-0.12,-0.12,-0.69
2,1872,-1.50,-0.94,-0.99,-0.35,-1.05,-1.18,-1.15,-1.05,-2.06,-1.44,-1.90,-1.68
3,1873,-1.44,-1.17,-1.53,-1.32,-0.98,0.71,0.23,0.00,0.86,0.38,-0.21,-0.91
4,1874,-0.83,-0.49,-0.75,-1.06,-1.23,0.11,-0.57,-1.09,-1.63,-1.76,-0.93,-1.12


### Cut down and mean

In [12]:
dates = []
for index, row in enso_df.iterrows():
    dates.append(f'{int(row.Year)}')
dates = pandas.to_datetime(dates)
enso_df['date'] = dates
enso_df.set_index('date', inplace=True)
# Remove Year
enso_df.drop(columns='Year', inplace=True)
enso_df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
date,,,,,,,,,,,,
1870-01-01,-0.94,-0.80,-0.25,-0.76,-0.91,-0.49,-0.75,-1.07,-0.26,-0.26,0.03,0.23
1871-01-01,0.39,-0.28,0.52,-0.16,-0.22,-0.45,-0.46,-0.39,-0.33,-0.12,-0.12,-0.69
1872-01-01,-1.50,-0.94,-0.99,-0.35,-1.05,-1.18,-1.15,-1.05,-2.06,-1.44,-1.90,-1.68
1873-01-01,-1.44,-1.17,-1.53,-1.32,-0.98,0.71,0.23,0.00,0.86,0.38,-0.21,-0.91
1874-01-01,-0.83,-0.49,-0.75,-1.06,-1.23,0.11,-0.57,-1.09,-1.63,-1.76,-0.93,-1.12


In [19]:
avgs = []
for index, row in enso_df.iterrows():
    avg = np.mean([row[key] for key in months])
    avgs.append(avg)
avgs = np.array(avgs)

In [24]:
cut_enso = pandas.DataFrame()
cut_enso['Mean'] = avgs[46:]
cut_enso.index = enso_df.index[46:]
cut_enso.head()

,Mean
date,
1916-01-01,-0.827500
1917-01-01,-1.134167
1918-01-01,0.068333
1919-01-01,0.975833
1920-01-01,0.169167


## SIO

In [32]:
SIO_file = '../Data/SIO_TEMP_1916_201905.csv'
SIO_df = pandas.read_csv(SIO_file, skiprows=26)
SIO_df.head()

,YEAR,MONTH,DAY,TIME_PST,TIME_FLAG,SURF_TEMP_C,SURF_FLAG,BOT_TEMP_C,BOT_FLAG,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1916.0,8.0,22.0,NaN,NaN,19.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1916.0,8.0,23.0,NaN,NaN,19.9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1916.0,8.0,24.0,NaN,NaN,19.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1916.0,8.0,25.0,NaN,NaN,19.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1916.0,8.0,26.0,NaN,NaN,19.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cut the last line

In [30]:
cut = np.ones(len(SIO_df), dtype=bool)
cut[-1] = False
cut_SIO = SIO_df.loc[cut].copy()
cut_SIO.iloc[-1]

year    2019.000
SST       18.682
Name: 103, dtype: float64

# pyextremes is *not* setup to handle a varying location, scale or shape..

In [ ]:
----